## <b><font color='darkblue'>What is a Plugin?</font></b>
([source](https://google.github.io/adk-docs/plugins/)) <font size='3ptx'><b>A Plugin in [Agent Development Kit](https://google.github.io/adk-docs/) (ADK) is a custom code module that can be executed at various stages of an agent workflow lifecycle using callback hooks.</b> You use Plugins for functionality that is applicable across your agent workflow. Some typical applications of Plugins are as follows:</font>

* <b><font size='3ptx'>Logging and tracing</font></b>: Create detailed logs of agent, tool, and generative AI model activity for debugging and performance analysis.
* <b><font size='3ptx'>Policy enforcement</font></b>: Implement security guardrails, such as a function that <b>checks if users are authorized to use a specific tool and prevent its execution if they do not have permission</b>.
* <b><font size='3ptx'>Monitoring and metrics</font></b>: Collect and export metrics on token usage, execution times, and invocation counts to monitoring systems such as Prometheus or [**Google Cloud Observability**](https://cloud.google.com/stackdriver/docs) (formerly Stackdriver)
* <b><font size='3ptx'>Response caching</font></b>: Check if a request has been made before, so you can return a cached response, skipping expensive or time consuming AI model or tool calls.
* <b><font size='3ptx'>Request or response modification</font></b>: Dynamically add information to AI model prompts or standardize tool output responses.

<b><font color='darkred'>Caution</font></b>: Plugins are not supported by the [ADK web interface](https://google.github.io/adk-docs/evaluate/#1-adk-web-run-evaluations-via-the-web-ui). If your ADK workflow uses Plugins, you must run your workflow without the web interface.

In [4]:
from IPython.display import display, Markdown, Latex
import warnings

warnings.filterwarnings('ignore')

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>How do Plugins work?</font></b>
([source](https://google.github.io/adk-docs/plugins/#how-do-plugins-work)) <font size='3ptx'><b>An ADK Plugin extends the <font color='blue'>BasePlugin</font> class and contains one or more `callback` methods, indicating where in the agent lifecycle the Plugin should be executed.</b> You integrate Plugins into an agent by registering them in your agent's `Runner` class. For more information on how and where you can trigger Plugins in your agent application, see [**Plugin callback hooks**](https://google.github.io/adk-docs/plugins/#plugin-callback-hooks).</font>

<b><font size='3ptx'>Plugin functionality builds on [Callbacks](https://google.github.io/adk-docs/callbacks/), which is a key design element of the ADK's extensible architecture</font></b>. While a typical Agent Callback is configured on a single agent, a single tool for a specific task, a Plugin is registered once on the `Runner` and its <b>callbacks apply globally to every agent, tool, and LLM call managed by that runner</b>.

### <b><font color='darkgreen'>Define and register Plugins</font></b>
([source](https://google.github.io/adk-docs/plugins/#define-and-register-plugins)) <font size='3ptx'><b>This section explains how to define Plugin classes and register them as part of your agent workflow. </b>For a complete code example, see [**Plugin Basic**](https://github.com/google/adk-python/tree/main/contributing/samples/plugin_basic) in the repository.</font>

#### <b>Create Plugin class</b>
Start by extending the <b><font color='blue'>BasePlugin</font></b> class and add one or more callback methods, as shown in the following code example:

In [5]:
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin

class CountInvocationPlugin(BasePlugin):
  """A custom plugin that counts agent and tool invocations."""

  def __init__(self) -> None:
    """Initialize the plugin with counters."""
    super().__init__(name="count_invocation")
    self.agent_count: int = 0
    self.tool_count: int = 0
    self.llm_request_count: int = 0

  async def before_agent_callback(
      self, *, agent: BaseAgent, callback_context: CallbackContext
  ) -> None:
    """Count agent runs."""
    self.agent_count += 1
    print(f"[Plugin] Agent run count: {self.agent_count}")

  async def before_model_callback(
      self, *, callback_context: CallbackContext, llm_request: LlmRequest
  ) -> None:
    """Count LLM requests."""
    self.llm_request_count += 1
    print(f"[Plugin] LLM request count: {self.llm_request_count}")

This example code implements callbacks for `before_agent_callback` and `before_model_callback` to count execution of these tasks during the lifecycle of the agent.

#### <b>Register Plugin class</b>
Integrate your Plugin class by registering it during your agent initialization as part of your `Runner` class, using the <font color='violet'>plugins</font> parameter. 

<b>You can specify multiple Plugins with this parameter</b>. The following code example shows how to register the <b><font color='blue'>CountInvocationPlugin</font></b> plugin defined in the previous section with a simple ADK agent.

In [6]:
from google.adk.runners import InMemoryRunner
from google.adk import Agent
from google.adk.tools.tool_context import ToolContext
from google.genai import types
import asyncio

async def hello_world(tool_context: ToolContext, query: str):
  print(f'Hello world: query is [{query}]')


root_agent = Agent(
    model='gemini-2.0-flash',
    name='hello_world',
    description='Prints hello world with user query.',
    instruction="Use `hello_world` tool to print hello world and user query.",
    tools=[hello_world],
)

In [7]:
async def main():
  """Main entry point for the agent."""
  prompt = 'hello world'
  runner = InMemoryRunner(
      agent=root_agent,
      app_name='test_app_with_plugin',
      # Add your plugin here. You can add multiple plugins.
      plugins=[CountInvocationPlugin()],
  )

  # The rest is the same as starting a regular ADK runner.
  session = await runner.session_service.create_session(
      user_id='user',
      app_name='test_app_with_plugin',
  )

  async for event in runner.run_async(
      user_id='user',
      session_id=session.id,
      new_message=types.Content(
        role='user', parts=[types.Part.from_text(text=prompt)]
      )
  ):
    print(f'** Got event from {event.author}')

#### <b>Run the agent with the Plugin</b>
<b><font size='3ptx'>Plugins are not supported by the [ADK web interface](https://google.github.io/adk-docs/evaluate/#1-adk-web-run-evaluations-via-the-web-ui)</font></b>. If your ADK workflow uses Plugins, you must run your workflow without the web interface. For more information on running ADK agents, see the [**Quickstart**](https://google.github.io/get-started/quickstart/#run-your-agent) guide.

In [8]:
await main()

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


[Plugin] Agent run count: 1
[Plugin] LLM request count: 1


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


** Got event from hello_world
Hello world: query is [hello world]
** Got event from hello_world
[Plugin] LLM request count: 2
** Got event from hello_world


### <b><font color='darkgreen'>Build workflows with Plugins</font></b>
([source](https://google.github.io/adk-docs/plugins/#build-workflows-with-plugins)) <font size='3ptx'><b>Plugin callback hooks are a mechanism for implementing logic that intercepts, modifies, and even controls the agent's execution lifecycle.</b> Each hook is a specific method in your Plugin class that you can implement to run code at a key moment. You have a choice between two modes of operation based on your hook's return value:</font>

* <b><font size='3ptx'>To Observe</font></b>: Implement a hook with no return value (`None`). This approach is for tasks such as logging or collecting metrics, as it allows the agent's workflow to proceed to the next step without interruption. For example, you could use `after_tool_callback` in a Plugin to log every tool's result for debugging.
* <b><font size='3ptx'>To Intervene</font></b>: Implement a hook and return a value. <b>This approach short-circuits the workflow</b>. The `Runner` halts processing, skips any subsequent plugins and the original intended action, <b>like a Model call, and use a Plugin callback's return value as the result</b>. A common use case is implementing `before_model_callback` to return a cached <b><font color='blue'>LlmResponse</font></b>, preventing a redundant and costly API call.
* <b><font size='3ptx'>To Amend</font></b>: Implement a hook and modify the `Context` object. <b>This approach allows you to modify the context data for the module to be executed without otherwise interrupting the execution of that module</b>. For example, adding additional, standardized prompt text for Model object execution.

<b><font color='orange'>Caution</font></b>:
> Plugin callback functions have precedence over callbacks implemented at the object level. This behavior means that Any Plugin callbacks code is executed before any Agent, Model, or Tool objects callbacks are executed. Furthermore, <b>if a Plugin-level agent callback returns any value, and not an empty (`None`) response, the Agent, Model, or Tool-level callback is not executed (skipped)</b>.

* <b><font size='3ptx'>Scope</font></b>: Plugin hooks are global. <b>You register a Plugin once on the `Runner`, and its hooks apply universally to every Agent, Model, and Tool it manages</b>. In contrast, Agent Callbacks are local, configured individually on a specific agent instance.
* <b><font size='3ptx'>Execution Order</font></b>: Plugins have precedence. <b>For any given event, the Plugin hooks always run before any corresponding Agent Callback</b>. This system behavior makes Plugins the correct architectural choice for implementing cross-cutting features like security policies, universal caching, and consistent logging across your entire application.

#### <b>Agent Callbacks and Plugins</b>
As mentioned in the previous section, there are some functional similarities between Plugins and Agent Callbacks. The following table compares the differences between Plugins and Agent Callbacks in more detail:

| | Plugins | Agent Callbacks |
| :--- | :--- | :--- |
| **Scope** | **Global:** Apply to all agents/tools/LLMs in the `Runner`. | **Local:** Apply only to the specific agent instance they are configured on. |
| **Primary Use Case** | **Horizontal Features:** Logging, policy, monitoring, global caching. | **Specific Agent Logic:** Modifying the behavior or state of a single agent. |
| **Configuration** | Configure once on the `Runner`. | Configure individually on each `BaseAgent` instance. |
| **Execution Order** | Plugin callbacks run **before** Agent Callbacks. | Agent callbacks run **after** Plugin callbacks. |

### <b><font color='darkgreen'>Plugin callback hooks</font></b>
([source](https://google.github.io/adk-docs/plugins/#plugin-callback-hooks)) <font size='3ptx'><b>You define when a Plugin is called with the callback functions to define in your Plugin class.</b> Callbacks are available when a user message is received, before and after an `Runner`, `Agent`, `Model`, or `Tool` is called, for `Events`, and when a `Model`, or `Tool` error occurs. These callbacks include, and take precedence over, the any callbacks defined within your `Agent`, `Model`, and `Tool` classes.</font>

The following diagram illustrates callback points where you can attach and run Plugin functionality during your agents workflow:
![plugin position in the working flow](https://google.github.io/adk-docs/assets/workflow-plugin-hooks.svg)
<br/><b>Figure 1</b>. Diagram of ADK agent workflow with Plugin callback hook locations.

The following sections describe the available callback hooks for Plugins in more detail:
* [**User Message callbacks**](#User-Message-callbacks)
* [**Runner start callbacks**](#Runner-start-callbacks)
* [**Agent execution callbacks**](#Agent-execution-callbacks)
* [**Model callbacks**](#Model-callbacks)
* [**Tool callbacks**](#Tool-callbacks)
* [**Events callbacks**](#Event-callbacks)
* [**Runner end callbacks**](#Runner-end-callbacks)

#### <b>User Message callbacks</b>
<b><font size='3ptx'>A User Message callback (`on_user_message_callback`) happens when a user sends a message</font></b>. The `on_user_message_callback` is the very first hook to run, giving you a chance to inspect or modify the initial input:
* <b><font size='3ptx'>When It Runs</font></b>: This callback happens immediately after `runner.run()`, before any other processing.
* <b><font size='3ptx'>Purpose</font></b>: The first opportunity to inspect or modify the user's raw input.
* <b><font size='3ptx'>Flow Control</font></b>: Returns a <b><font color='blue'>types.Content object</font></b> to replace the user's original message.

The following code example shows the basic syntax of this callback:
```python
async def on_user_message_callback(
    self,
    *,
    invocation_context: InvocationContext,
    user_message: types.Content,
) -> Optional[types.Content]:
```

#### <b>Runner start callbacks</b>
<b><font size='3ptx'>A Runner start callback (`before_run_callback`) happens when the `Runner` object takes the potentially modified user message and prepares for execution.</font></b> The `before_run_callback` fires here, allowing for global setup before any agent logic begins.
* <b><font size='3ptx'>When It Runs</font></b>: Immediately after `runner.run()` is called, before any other processing.
* <b><font size='3ptx'>Purpose</font></b>: The first opportunity to inspect or modify the user's raw input.
* <b><font size='3ptx'>Flow Control</font></b>: Return a <b><font color='blue'>types.Content object</font></b> to replace the user's original message.

The following code example shows the basic syntax of this callback:
```python
async def before_run_callback(
    self, *, invocation_context: InvocationContext
) -> Optional[types.Content]:
```

#### <b>Agent execution callbacks</b>
<b><font size='3ptx'>Agent execution callbacks (`before_agent`, `after_agent`) happen when a `Runner` object invokes an agent.</font></b> The `before_agent_callback` runs immediately before the agent's main work begins. The main work encompasses the agent's entire process for handling the request, which could involve calling models or tools. After the agent has finished all its steps and prepared a result, the `after_agent_callback` runs.

<b><font color='orange'>Caution</font></b>:
> Plugins that implement these callbacks are executed before the Agent-level callbacks are executed. Furthermore, if a Plugin-level agent callback returns anything other than a `None` or null response, the Agent-level callback is not executed (skipped).

For more information about Agent callbacks defined as part of an Agent object, see [**Types of Callbacks**](https://google.github.io/adk-docs/callbacks/types-of-callbacks/#agent-lifecycle-callbacks).

#### <b>Model callbacks</b>
<b><font size='3ptx'>Model callbacks (`before_model`, `after_model`, `on_model_error`) happen before and after a Model object executes.</font></b> The Plugins feature also supports a callback in the event of an error, as detailed below:
* If an agent needs to call an AI model, `before_model_callback` runs first.
* If the model call is successful, `after_model_callback` runs next.
* If the model call fails with an exception, the `on_model_error_callback` is triggered instead, allowing for graceful recovery.

<b><font color='orange'>Caution</font></b>:
> Plugins that implement the `before_model` and `after_model` callback methods are executed before the Model-level callbacks are executed. Furthermore, if a Plugin-level model callback returns anything other than a `None` or null response, the Model-level callback is not executed (skipped).

##### <b>Model on error callback details</b>
The `on_model_error` for Model objects is only supported by the Plugins feature works as follows:
* <b><font size='3ptx'>When It Runs</font></b>: When an exception is raised during the model call.
* <b><font size='3ptx'>Common Use Cases</font></b>: Graceful error handling, logging the specific error, or returning a fallback response, such as "`The AI service is currently unavailable.`"
* <b><font size='3ptx'>Flow Control</font></b>:
    - Returns an <b><font color='blue'>LlmResponse</font></b> object to suppress the exception and provide a fallback result.
    - Returns `None` to allow the original exception to be raised.

<b><font color='darkred'>Note</font></b>: If the execution of the Model object returns a <b><font color='blue'>LlmResponse</font></b>, the system resumes the execution flow, and `after_model_callback` will be triggered normally.

The following code example shows the basic syntax of this callback:
```python
async def on_model_error_callback(
    self,
    *,
    callback_context: CallbackContext,
    llm_request: LlmRequest,
    error: Exception,
) -> Optional[LlmResponse]:
  ...
```

#### <b>Tool callbacks</b>
<b><font size='3ptx'>Tool callbacks (`before_tool`, `after_tool`, `on_tool_error`) for Plugins happen before or after the execution of a tool, or when an error occurs.</font></b> The Plugins feature also supports a callback in the event of an error, as detailed below:
When an agent executes a Tool, before_tool_callback runs first.
* If the tool executes successfully, `after_tool_callback` runs next.
* If the tool raises an exception, the `on_tool_error_callback` is triggered instead, giving you a chance to handle the failure.
* If `on_tool_error_callback` returns a dict, `after_tool_callback` will be triggered normally.

<b><font color='orange'>Caution</font></b>:
> Plugins that implement these callbacks are executed before the Tool-level callbacks are executed. Furthermore, if a Plugin-level tool callback returns anything other than a `None` or null response, the Tool-level callback is not executed (skipped).

##### <b>Tool on error callback details</b>
The on error callback for Tool objects is only supported by the Plugins feature works as follows:
* <b><font size='3ptx'>When It Runs</font></b>: When an exception is raised during the execution of a tool's run method.
* <b><font size='3ptx'>Purpose</font></b>: Catching specific tool exceptions (like `APIError`), logging the failure, and providing a user-friendly error message back to the LLM.
* <b><font size='3ptx'>Flow Control</font></b>: Return a `dict` to suppress the exception, provide a fallback result. Return `None` to allow the original exception to be raised.

<b><font color='darkred'>Note</font></b>: By returning a `dict`, this resumes the execution flow, and `after_tool_callback` will be triggered normally.

The following code example shows the basic syntax of this callback:
```python
async def on_tool_error_callback(
    self,
    *,
    tool: BaseTool,
    tool_args: dict[str, Any],
    tool_context: ToolContext,
    error: Exception,
) -> Optional[dict]:
  ...
```

#### <b>Event callbacks</b>
<b><font size='3ptx'>An Event callback (`on_event_callback`) happens when an agent produces outputs such as a text response or a tool call result, it yields them as `Event` objects</font></b>. The `on_event_callback` fires for each event, allowing you to modify it before it's streamed to the client.
* <b><font size='3ptx'>When It Runs</font></b>: After an agent yields an Event but before it's sent to the user. An agent's run may produce multiple events.
* <b><font size='3ptx'>Purpose</font></b>: Useful for modifying or enriching events (<font color='brown'>e.g., adding metadata</font>) or for triggering side effects based on specific events.
* <b><font size='3ptx'>Flow Control</font></b>: Return an `Event` object to replace the original event.

The following code example shows the basic syntax of this callback:
```python
async def on_event_callback(
    self, *, invocation_context: InvocationContext, event: Event
) -> Optional[Event]:
  ...
```

#### <b>Runner end callbacks</b>
<b><font size='3ptx'>The Runner end callback (`after_run_callback`) happens when the agent has finished its entire process and all events have been handled, the `Runner` completes its run.</font></b> The `after_run_callback` is the final hook, perfect for cleanup and final reporting.
* <b><font size='3ptx'>When It Runs</font></b>: After the Runner fully completes the execution of a request.
* <b><font size='3ptx'>Purpose</font></b>: Ideal for global cleanup tasks, such as closing connections or finalizing logs and metrics data.
* <b><font size='3ptx'>Flow Control</font></b>: This callback is for teardown only and cannot alter the final result.

The following code example shows the basic syntax of this callback:
```python
async def after_run_callback(
    self, *, invocation_context: InvocationContext
) -> Optional[None]:
  ...
```